# Bayesian Belief Network #

In [16]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch, BicScore 
from pgmpy.inference import VariableElimination

df = pd.read_csv("encoded_df.csv")

df.head()

,id,date,minute,rating,relDate,relCountry_Argentina,relCountry_Australia,relCountry_Brazil,relCountry_Canada,relCountry_China,...,language_Spanish,language_Swedish,language_Tagalog,language_Tamil,language_Telugu,language_Thai,language_Turkish,language_Ukrainian,language_Urdu,language_Vietnamese
0,1000001,2023.0,114.0,3.86,2023-07-06,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000004,1999.0,139.0,4.27,1999-09-10,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000006,2023.0,181.0,4.23,2023-07-11,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000008,2019.0,122.0,3.85,2019-08-31,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000011,2018.0,117.0,4.42,2018-12-01,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Manually define the structure of the BBN
model = BayesianNetwork([("date", "minute"), ("rating", "minute")])

# Fit the BBN model
model.fit(df, estimator = MaximumLikelihoodEstimator)
print(model.edges())


[('date', 'minute'), ('rating', 'minute')]


In [18]:
# Alternative model structure found using a scoring method
# hill_climb = HillClimbSearch(df)
# hc_model = hill_climb.estimate(scoring_method = BicScore(df))

# Current encoded_df is too large/not encoded in such a way that this method is conducive. 

In [ ]:
# Perform inference on the chosen BBN model
inference = VariableElimination(model)

# Query the network for probability of given variables
q_result_1 = inference.query(variables = ["minute"], evidence = {"rating": 4.23})
print(q_result_1)

q_result_2 = inference.query(variables = ["rating"], evidence = {"minute": 122.0})
print(q_result_2)

+---------------+---------------+
| minute        |   phi(minute) |
+===============+===============+
| minute(1.0)   |        0.0036 |
+---------------+---------------+
| minute(2.0)   |        0.0036 |
+---------------+---------------+
| minute(3.0)   |        0.0036 |
+---------------+---------------+
| minute(4.0)   |        0.0036 |
+---------------+---------------+
| minute(5.0)   |        0.0036 |
+---------------+---------------+
| minute(6.0)   |        0.0036 |
+---------------+---------------+
| minute(7.0)   |        0.0036 |
+---------------+---------------+
| minute(8.0)   |        0.0036 |
+---------------+---------------+
| minute(9.0)   |        0.0036 |
+---------------+---------------+
| minute(10.0)  |        0.0036 |
+---------------+---------------+
| minute(11.0)  |        0.0036 |
+---------------+---------------+
| minute(12.0)  |        0.0036 |
+---------------+---------------+
| minute(13.0)  |        0.0036 |
+---------------+---------------+
| minute(14.0)